In our last exploring NLP notebook we built an email spam detector using Natural Language Processing techniques and the Support Vector Machine (SVM) algorithm for classification.
In this project, we will again build a spam detector but this time using URLs instead of emails.

In [8]:
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
import nltk

from nltk.corpus import stopwords
import unicodedata

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection, svm
from sklearn.metrics import classification_report, accuracy_score

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [14]:
df.sample(10)

,url,is_spam
2126,https://www.doordash.com/,False
2577,https://docs.google.com/forms/d/e/1FAIpQLSew3g...,True
2904,https://www.si.com/sports-illustrated/2020/06/...,False
2625,https://www.nytimes.com/2020/06/29/climate/hou...,False
1029,https://www.youtube.com/watch?v=OqRWb9JQW_0,False
378,https://briefingday.us8.list-manage.com/unsubs...,True
175,https://www.npr.org/2020/05/20/858347477/money...,False
1494,https://www.theguardian.com/commentisfree/2020...,False
2884,https://hyperapp.dev/,False
1077,https://hirshhorn.si.edu/hirshhorninsideout/,False


In [19]:
df['is_spam'] = df['is_spam'].apply(lambda x: 1 if x == True else 0) #columna categorica la pasamos a 0 y 1

In [20]:
print("Cant spam: " +str(len(df.loc[df.is_spam==1])))
print("Cant q no es spam: " +str(len(df.loc[df.is_spam==0])))
print(df.shape)
#df['is_spam'] = df['is_spam'].astype(int) #otra forma!

Cant spam: 0
Cant q no es spam: 2999
(2999, 2)


In [21]:
df['url'] = df['url'].str.lower() 


In [25]:
stop=stopwords.words('english')

def remove_stopwords(message):
    if message is not None:
        words = message.strip().split()
        words_filtered = []
        for word in words:
            if word not in stop:
                words_filtered.append(word)
        result = " ".join(words_filtered)  
    else:
        result=None
    return result

def clean_text_digits(texto):
    #'''Match all digits in the string and replace them by empty string.'''
    if texto is not None:
        pattern1 = r'[0-9]'
        pattern2 = '[^a-zA-Z]'
        pattern3 = "(\\d|\\W)+"
        pattern4 = r'http(s)'
        result = re.sub(pattern1, '', texto)
        result = re.sub(pattern2, '', result)
        result = re.sub(pattern3, '', result)
        result = re.sub(pattern4, '', result)
    else:
        result=None
    return result

#df['url']=df['url'].apply(remove_stopwords)
df['url']=df['url'].apply(clean_text_digits)

#df['url'] = df['url'].str.replace(r'http(s)', '', regex=True)

In [26]:
df['url'] 

0                   briefingdayuslistmanagecomunsubscribe
1                                             wwwhvpercom
2                                     briefingdaycommvnif
3                             briefingdaycomnmcommentform
4                                       briefingdaycomfan
                              ...                        
2994    wwwsmartcitiesworldnetnewsnewsdeepfaketechnolo...
2995                                   wwwyoutubecomwatch
2996             techcrunchcomanoptimisticviewofdeepfakes
2997    wwwtechnologyreviewcomthisstartupclaimsitsdeep...
2998                              wwwbbccomnewstechnology
Name: url, Length: 2999, dtype: object

In [ ]:
#limpio
def normalize_string(text_string):
    if text_string is not None:
        result = unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result = None
    return result

In [ ]:
df['len_url'] = df['url'].apply(lambda x : len(x))
df['contains_subscribe'] = df['url'].apply(lambda x : 1 if "subscribe" in x else 0)
df['contains_hash'] = df['url'].apply(lambda x : 1 if "#" in x else 0)
df['num_digits'] = df['url'].apply(lambda x : len("".join(_ for _ in x if _.isdigit())) )
df['non_https'] = df['url'].apply(lambda x : 1 if "https" in x else 0)
df['num_words'] = df['url'].apply(lambda x : len(x.split("/")))

target = 'is_spam'
features = [f for f in df.columns if f not in ["url", target]]
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=0)